In [141]:
#!/usr/bin/env python
# coding: utf-8
from qiskit import*
import numpy as np
from sympy.combinatorics import GrayCode
#from numpy.linalg import matrix_power

In [73]:
Target_state = '0000'

In [74]:
# First we note the length of N.
L = len(Target_state)

N = 2**L

# Then an identity matrix is created with the size 2**N with signs reversed.
U_w = - np.identity(2 ** L, dtype=complex)


# Then the sign of the element corresponding to the target state is flipped. To do that we first convert the
# target state from binary to decimal number. 
Target_index = int(Target_state, 2)

## The position of the target state is set as 1.
U_w.itemset((Target_index, Target_index),1)

In [75]:
## We will first create a matrix with all elements 1. This is |psi><psi| = A(say).
A = np.ones((2**L, 2**L))

## U_s = 2\(2**N)2|psi><psi| - I
U_s = (2/(2**L))*A - np.identity(2**L)

## G is the Grover operator.
G = np.matmul(U_w, U_s)


#print('The Grover operator for the target state |w > = | '+Target_state + ' > is \n\n',G.real)

In [143]:
## The dimension of the matrix is fixed by the number of qubits.
def MCT(List,c,N):
    ## Changing the simulator 
    backend = Aer.get_backend('unitary_simulator')

    ## The circuit without measurement
    circ = QuantumCircuit(N)
    circ.mct(List,c)

    ## job execution and getting the result as an object
    job = execute(circ, backend)
    result = job.result()

    ## get the unitary matrix from the result object
    return result.get_unitary(circ) 

In [144]:
G = np.matrix(MCT([0,1,2],3,4))

The operator we will unfold into two qubit gates is $$ G = (2( |\psi><\psi|-I)O)^M $$

In [145]:
#G = matrix_power(V,Num_iteration)

The following function calculates two level unitary matrices for a matrix of size
greater than 3, returns a ordered pair of list
 of matrices U and the final product of all the U matrices.
 Following the algorithm given in Nielsen, M.A. and Chuang.

In [146]:
def ReductionN(Matrix):

    Two_Level_Universal_Gate = []
    
    # Extract the size of the input matrix.
    Matrix_dim = int(np.sqrt(Matrix.size))

    for i in range(1,Matrix_dim):

        if Matrix[i,0] == 0:

            U = np.identity(Matrix_dim, dtype = complex)

            if i == 1:
                pass
            else:
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(Matrix_dim, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating the product of all the U matrices with G.

        Matrix = np.matmul(U, Matrix)

    return Two_Level_Universal_Gate, Matrix

Function calculates two level unitary matrices for a 3x3 matrix.
    Returns a list containing the two level matrices.
    The function will be called reduced as it put zeros in the first 
    column and the first row.In what follows, the final matrix product
    will be called reduced matrix.

In [147]:
def Reduction3(Matrix):

    Two_Level_Universal_Gate = []
    
    # Size of the matrix is 3.
    Matrix_dim = 3
 
    for i in range(1,Matrix_dim):

        '''
            If the (1,0) element is 0, then U_1 = identity matrix.
        
        '''
        
        if Matrix[i,0] == 0:

            U = np.identity(Matrix_dim, dtype = complex) # U_1 is identity.

            # For the first iteration it is identity.
            if i == 1:
                
                pass
            
            else:
                
                U.itemset((0,0), np.conjugate(Matrix[0,0]))

        else:

            a, b = Matrix[0,0], Matrix[i,0]

            u00 = np.conjugate(a) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u01 = np.conjugate(b) / np.sqrt(np.absolute(a)**2 \
                + np.absolute(b)**2)
            u10 = b / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
            u11 = - a / np.sqrt(np.absolute(a)**2 + np.absolute(b)**2)
    
            U = np.identity(Matrix_dim, dtype = complex)
            U.itemset((0,0), u00)
            U.itemset((0,i), u01)
            U.itemset((i,0), u10)
            U.itemset((i,i), u11)
        
        Two_Level_Universal_Gate.append(U)

        # Calculating product of all the U matrices with G.
        Matrix = np.matmul(U, Matrix)

    # U3 is calculated sepatarely.
    Two_Level_Universal_Gate.append(np.conjugate(Matrix).transpose())

    return Two_Level_Universal_Gate

The following function returns a sub matrix with one dimension lower than the input matrix.

In [148]:
def Extract_subMatrix(Matrix):

    Matrix_size = int(np.sqrt(Matrix.size))
    subMatrix = Matrix[1:Matrix_size, 1:Matrix_size]

    return  subMatrix

The following function expands a given matrix to a required size.

In [149]:
def Expand_Matrix(Matrix, Required_Size):

    # Intital size of the input matrix.
    Initial_Size = int(np.sqrt(Matrix.size))

    # Creating an initial identity matrix.
    M = np.identity(Required_Size, dtype = complex)

    # Size difference between the two matrics.
    Sz = Required_Size - Initial_Size

    # The elements from the old matrix is added to M, other row and column are part of
    # identity matrix, this gurantees that the matrix is a two level unitary matrix.

    for i in range(Sz, Required_Size):
        
        for j in range(Sz, Required_Size):
            
            M.itemset((i,j), Matrix[i-Sz,j-Sz])

    return M

In [150]:
N

16

This loop will calculate the two level unitary matrices using the functions described above.

In [151]:
def Reduce(Matrix):
    # This list will contain all the two level unitary gates.
    Unitary_gates = []

    Last_Reduced_Matrix = []

    for i in range(N, 2, -1): # Loop from N to 3.
    
    
        # 3x3 matrices are treated separetely with CReduction3(Matrix).
        if i == 3:
        
            M = Last_Reduced_Matrix[-1]

            for g in Reduction3(M):
            
                Unitary_gates.append(g)

        # First iteration, start with original matrix G.
        elif i == N:

            M = Matrix
            
            for g in ReductionN(M)[0]: # List of two level gates.
            
                Unitary_gates.append(g)

        
            # The reduced submatrix is put into the list to use in the next loop.
            Last_Reduced_Matrix.append(Extract_subMatrix(ReductionN(M)[1]))
            #print(Last_Reduced_Matrix)
        
        else:
        
            # Any other dimension is reduced with CReductionN(Matrix).
            M = Last_Reduced_Matrix[-1]

            for g in ReductionN(M)[0]:
            
                Unitary_gates.append(g)   

            # The reduced submatrix is put into the list to use in the next loop.
            Last_Reduced_Matrix.append(Extract_subMatrix(ReductionN(M)[1]))        
            #print(Last_Reduced_Matrix)
    return Unitary_gates

Prints all the two level unitary gates.

In [152]:
def Unfold(Matrix):
    
    Gates = []
    i = 1
    for g in Reduce(Matrix):

        # If the matrix is of size NxN, print the matrix.
        if int(np.sqrt(g.size)) == N :
            
            Gates.append(["U"+str(i), g])
            #print('U'+str(i)+'\n', g,'\n\n')

        else:

            # If the matrix is of lower size other than N, expand the matrix into a NxN
            # matrix and print.
            
            Gates.append(["U"+str(i), Expand_Matrix(g, N) ])
            #print('U'+str(i)+'\n', Expand_Matrix(g, N),'\n\n')

        i += 1
        
    return Gates

In [156]:
K = Unfold(G)
F = np.identity(N, dtype=complex)
for i in range(len(K)):
    print('U'+str(i),'\n', K[i][1].real,'\n')
    #F = np.matmul(F,np.matrix(K[i][1]).getH())

U0 
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]] 

U1 
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0

# Code below generates Gray Code

In [154]:
'''
The function takes a two level unitary matrix as input and calculates the vector components
where it acts non-trivially. It finally returns the Gray Code of the index of two vector
components. (Reference - Neilsen and Chuang)

'''
def Corresponding_Gray_Code(U_Matrix):
    
    
    '''
    
    Function retuens the row index of the non-zero elements of a given matrix.
    
    '''
    def Non_Zero_elements_Indices(U_Matrix):
        
        Indices_Array = np.nonzero(U_Matrix-np.identity(N))
        row_index = Indices_Array[0]
        column_index = Indices_Array[1]
    
        #Index_Pair = []
        #for i in range(len(row_index)):
            #Index_Pair.append((row_index[i],column_index[i]))
                              
        return row_index    
    
    
    
    def Dec2Bin(DecimalNumber): # Converts decimal to binary numbers.
        return bin(DecimalNumber).replace("0b", "")


    '''
    
    Functions returns the initial element g1 and the final element gm as binary string.
    
    '''
    def s_t(Array):
    
        List = [Array[0], Array[-1]]

    
        l = []
    
        for i in List:
        
            i_Bin = Dec2Bin(i)
              
        
            '''
            While converting numbers from decimal to binary, for example, 1 is mapped to 1, to make sure that
            every numbers have N qubits in them, the following loop adds leading zeros to make the
            length of the binary string equal to N. Now, 1 is mapped to 000.....1 (string of length N).
        
            '''
        
            while len(i_Bin) < L: 
            
                i_Bin = '0'+i_Bin # This loop adds leading zeros.
            
            l.append(i_Bin)
        
        return l    

    
        def Gray_Code(List):
            
            a = GrayCode(L)
            Gray_Code_List = list(a.generate_gray())
            i1, i2 = Gray_Code_List.index(List[0]), Gray_Code_List.index(List[1])
            
            l = []
            for i in range(i1,i2+1):
                l.append(Gray_Code_List[i])
            return l
            
    return Gray_Code(s_t(Non_Zero_elements_Indices(U_Matrix)))               

In [155]:
Corresponding_Gray_Code(K[5][1])

IndexError: index 0 is out of bounds for axis 0 with size 0